In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read data
df_raw = pd.read_csv('experiment_output_of_improved_True.tsv', sep='\t', na_values='error').drop(columns=['phi', 'pmax', 'threads', 't_max_min'])
df_raw.replace('Plurality', 'plurality', inplace=True)
df_raw.head()

In [ ]:
df_unfinished = df_raw[df_raw['t_mpw_s'].isna()].sort_values(by=['m', 'n']).drop(columns=['batch', 'mew', 't_mew_s']).drop_duplicates()
df_unfinished.head()

In [ ]:
df_finished = df_raw.dropna()
df_finished = df_finished.groupby(['m', 'n', 'rule'], as_index=False).agg({'batch': 'count', 't_mpw_s': 'mean', 't_mew_s': 'mean'})
df_finished = df_finished.query('batch > 5')
df_finished.head()

In [ ]:
import matplotlib as mpl
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['pdf.fonttype'] = 42
mpl.rc('font', **{'size': 18})

In [ ]:
savemode = True

In [ ]:
df_unfinished_skyline = df_unfinished[['m', 'n']].copy()
df_unfinished_skyline['execution'] = 'out-of-30min'
df_finished_skyline = df_finished[['m', 'n']].copy()
df_finished_skyline['execution'] = 'finished'

df_skyline = pd.concat([df_finished_skyline, df_unfinished_skyline], ignore_index=True)

plt.figure(dpi=120)
sns.scatterplot(x='m', y='n', hue='execution', data=df_skyline)
plt.xticks(list(range(3, 14)))
# plt.yticks(list(range(2, 6)))
plt.xlabel('#cand')
plt.ylabel('#voters')
plt.legend(title='', handletextpad=0)
plt.title('Skyline of scalablity');

In [ ]:
df_keys = df_finished[['m', 'n', 'rule']].copy()
df_mpw = df_keys.copy()
df_mew = df_keys.copy()
df_mpw['definition'] = 'parallel MPW'
df_mew['definition'] = 'parallel MEW'
df_mpw['time(s)'] = df_finished['t_mpw_s'].copy()
df_mew['time(s)'] = df_finished['t_mew_s'].copy()
df_vis = pd.concat([df_mpw, df_mew])

In [ ]:
dfx = df_vis.query('m == 9')

plt.figure(dpi=120)
sns.pointplot(x='n', y='time(s)', hue='definition', data=dfx, ci=0)
plt.yscale('log')
# plt.ylim(0.0005, 100)
plt.xlabel('#voters')
# plt.xticks(list(range(0, 16, 2)))
plt.legend(title='', ncol=1, loc=2, 
           borderpad=0.2, labelspacing=0.2, handlelength=1, 
           handletextpad=0.2, borderaxespad=0.2, columnspacing=0.5);
plt.tight_layout(pad=0)
if savemode:
    plt.savefig('synthetic_mpw_parallel_pluarlity_9_candidates__time_vs_n.pdf')

In [ ]:
dfx = df_vis.query('n == 10')

plt.figure(dpi=120)
sns.pointplot(x='m', y='time(s)', hue='definition', data=dfx, ci=0)
plt.yscale('log')
# plt.ylim(0.02, 2000)
plt.xlabel('#cand')
# plt.xticks(list(range(0, 16, 2)))
plt.legend(title='', ncol=1, loc=2, 
           borderpad=0.2, labelspacing=0.2, handlelength=1, 
           handletextpad=0.2, borderaxespad=0.2, columnspacing=0.5);
plt.tight_layout(pad=0)
if savemode:
    plt.savefig('synthetic_mpw_parallel_pluarlity_10_voters__time_vs_m.pdf')